#### -Use several technical indicators to build features from price
#### -Use Random Forest model to extract a subset of features.
#### -Use Extreme Gradient Boosting Tree as a prediction model. 
#### -Rolling window is used in prediction and model is retrained every time
#### -The validation dataset has a trick in this strategy. It is used for the optimization of the prediction power with the feature selection of Random Forest  at every time point .

In [ ]:
## Import the library and data
import pymysql   
import pandas as pd
import numpy as np
from pyti.exponential_moving_average import exponential_moving_average as ema
from pyti.relative_strength_index import relative_strength_index as rsi
from pyti import stochastic as stoc
from pyti.williams_percent_r import williams_percent_r as wr
from pyti.rate_of_change import rate_of_change as roc
from pyti.commodity_channel_index import commodity_channel_index as cci
from pyti.simple_moving_average import simple_moving_average as sma
from pyti.momentum import momentum as mom
from pyti.volatility import volatility as vol
from pyti.average_true_range_percent import average_true_range_percent as atrp
from pyti.detrended_price_oscillator import detrended_price_oscillator as dpo
from pyti.chande_momentum_oscillator import chande_momentum_oscillator as cmo
from pyti.directional_indicators import average_directional_index as adi
from pyti.double_smoothed_stochastic import double_smoothed_stochastic as dss
from pyti.stochrsi import stochrsi
from arch import arch_model
import xgboost as xgb
from sklearn.metrics import confusion_matrix as table_matrix
import matplotlib.pyplot as plt
import sklearn.svm
from sklearn.ensemble import RandomForestClassifier as RFC



conn = pymysql.connect(host=<IP>,  
                       port=3306,  
                       user=<USER>,  
                       passwd=<PW>,  
                       db=<db>,  
                       charset='utf8')  


sql = "SELECT * FROM CU where ContractName='CU_Index';"  
CU = pd.read_sql(sql, conn)[['StrDateTime','OPEN','HIGH','LOW','LAST_PRICE']]
CU.columns=['times','open','high','low','close']


In [ ]:
#%% Functions 
def read_data(data,field):
    data_0=data
    data_0['times'] = pd.to_datetime(data_0.times)
    data_0 = data_0.sort_values(by='times')
    data_0.index=data_0['times'].values       
    #data_0.index=data_0['times']
    data_0=data_0[field].resample('1H',label='right').ohlc() #data frequency
    data_0['times']=data_0.index
    data_0.dropna(inplace=True)
    return data_0
def slope(data, period):
    """
    """
    from pyti import catch_errors
    from pyti.function_helper import fill_for_noncomputable_vals
    from six.moves import range    
    catch_errors.check_for_period_error(data, period)
    rocs =[((data[idx] - data[idx - (period - 1)]) / (period - 1))  for idx in range(period - 1, len(data))]
    rocs = fill_for_noncomputable_vals(data, rocs)
    return rocs

def rolling_garch(returns,n,p,o,q):
    vol=np.repeat(np.nan,n)
    for i in range(n,returns.shape[0]):
        x=returns[(i-n):(i)]
        am = arch_model(x, vol='Garch', p=p, o=o, q=q, dist='Normal').fit()
        vol=np.append(vol,am.conditional_volatility.values[-1])   
    return vol
        
def data_prep(dataset):
    ema_diff=ema(dataset['close'],12)-ema(dataset['close'],26)
    ema_diff_2=ema(dataset['close'],24)-ema(dataset['close'],75)
    returns=np.log(dataset['close']/dataset['close'].shift(1))   	
    dataset=dataset.assign(**{
                            'EMA_1':ema_diff,       
                            'EMA_2':ema(dataset['close'],30)-ema(dataset['close'],90), 
                            'EMA_3':ema(dataset['close'],20)-ema(dataset['close'],60), 
                            'EMA_4':ema(dataset['close'],40)-ema(dataset['close'],130),
                            'EMA_5':ema(dataset['close'],18)-ema(dataset['close'],54), 				   
                            'MACD_1':ema_diff - ema(ema_diff,9),
                            'MACD_2':(ema_diff_2)- ema(ema_diff_2,12),					   
                            'MOM_1':mom(dataset['close'],3),					    
                            'MOM_2':mom(dataset['close'],10),					   
                            'MOM_3':mom(dataset['close'],20),				       					    
                            'MOM_4':mom(dataset['close'],25),
                            'VOL_1':vol(dataset['close'],3),
                            'VOL_2':vol(dataset['close'],10),					    
                            'VOL_3':vol(dataset['close'],23),    
                            'VOL_4':vol(dataset['close'],35),					    
                            'RSI_1':rsi(dataset['close'],14),					    
                            'RSI_2':rsi(dataset['close'],12),					   
                            'RSI_3':rsi(dataset['close'],10),					    
                            'RSI_4':rsi(dataset['close'],8),					    
                            'RSI_5':rsi(dataset['close'],7),					    
                            'STOC_1':stoc.percent_k(dataset['close'],20)-stoc.percent_d(dataset['close'],20),					    
                            'STOC_2':stoc.percent_k(dataset['close'],40)-stoc.percent_d(dataset['close'],40),					     		
                            'STOC_3':stoc.percent_k(dataset['close'],33)-stoc.percent_d(dataset['close'],33),
                            'STOC_4':stoc.percent_k(dataset['close'],12)-stoc.percent_d(dataset['close'],12),					    
                            'STOC_5':stoc.percent_k(dataset['close'],25)-stoc.percent_d(dataset['close'],25),					    
                            'WR':wr(dataset['close']),					    
                            'ROC_1':roc(dataset['close'],7),					    	
                            'ROC_2':roc(dataset['close'],12),
                            'ROC_3':roc(dataset['close'],5),
                            'ROC_4':roc(dataset['close'],13),
                            'ROC_5':roc(dataset['close'],10),
                            'ATRP_1':atrp(dataset['close'],3),	                 					
                            'ATRP_2':atrp(dataset['close'],7),
                            'ATRP_3':atrp(dataset['close'],14),	                 
                            'ATRP_4':atrp(dataset['close'],30),	                 
                            'ATRP_5':atrp(dataset['close'],50),	                 
                            'CCI_1':cci(dataset['close'],dataset['high'],dataset['low'],20),	                 
                            'CCI_2':cci(dataset['close'],dataset['high'],dataset['low'],15),	                 
                            'CCI_3':cci(dataset['close'],dataset['high'],dataset['low'],7),	                 
                            'CCI_4':cci(dataset['close'],dataset['high'],dataset['low'],11),	                 
                            'CCI_5':cci(dataset['close'],dataset['hignh'],dataset['low'],10),	                 
                            'DPO_1':dpo(dataset['close'],5),	
                            'DPO_2':dpo(dataset['close'],10),
                            'DPO_3':dpo(dataset['close'],20),
                            'DPO_4':dpo(dataset['close'],30),
                            'DPO_5':dpo(dataset['close'],40),
                            'CMO_1':cmo(dataset['close'],5),
                            'CMO_2':cmo(dataset['close'],10),
                            'CMO_3':cmo(dataset['close'],20),
                            'CMO_4':cmo(dataset['close'],30),
                            'CMO_5':cmo(dataset['close'],40),
                            'ADI_1':adi(dataset['close'],dataset['high'],dataset['low'],5),
                            'ADI_2':adi(dataset['close'],dataset['high'],dataset['low'],10),
                            'ADI_3':adi(dataset['close'],dataset['high'],dataset['low'],20),
                            'ADI_4':adi(dataset['close'],dataset['high'],dataset['low'],30),
                            'ADI_5':adi(dataset['close'],dataset['high'],dataset['low'],40),
                            'DSS_1':dss(dataset['close'],5),  
                            'DSS_2':dss(dataset['close'],10),
                            'DSS_3':dss(dataset['close'],20),
                            'DSS_4':dss(dataset['close'],30),
                            'DSS_5':dss(dataset['close'],40),
                            'STOCHRSI_1':stochrsi(dataset['close'],5),
                            'STOCHRSI_2':stochrsi(dataset['close'],14),  
                            'STOCHRSI_3':stochrsi(dataset['close'],20),
                            'STOCHRSI_4':stochrsi(dataset['close'],30),
                            'STOCHRSI_5':stochrsi(dataset['close'],40),
                            'H_1':dataset['high'],
                            'H_2':sma(dataset['high'],5),
                            'H_3':sma(dataset['high'],15),
                            'L_1':sma(dataset['low'],1),
                            'L_2':sma(dataset['low'],5),
                            'L_3':sma(dataset['low'],15),
                            'slope_1':slope(dataset['high'],3),   
                            'slope_2':slope(dataset['high'],6),
                            'slope_3':slope(dataset['high'],9),
                            'slope_4':slope(dataset['high'],12),
                            'slope_5':slope(dataset['high'],15),
                            'slope_6':slope(dataset['high'],18),
                            'slope_7':slope(dataset['low'],3),
                            'slope_8':slope(dataset['low'],6),
                            'slope_9':slope(dataset['low'],9),
                            'slope_10':slope(dataset['low'],12),
                            'slope_11':slope(dataset['low'],15),
                            'slope_12':slope(dataset['low'],18),   
                            'returns':np.log(dataset['close']/dataset['close'].shift(1)),
                            'GARCH_1':rolling_garch(returns,20*8,1,0,1),
                            'GARCH_2':rolling_garch(returns,20*8,3,0,1),
                            'GARCH_3':rolling_garch(returns,20*8,2,0,1),
                            'label':(np.log(dataset['close']/dataset['close'].shift(1))).shift(-1)})
    #'GARCH_4':rolling_garch(returns,20*8,1,0,2)
    #'GARCH_5':rolling_garch(returns,20*8,1,0,3)
    #'GARCH_6':rolling_garch(returns,20*8,1,1,1)
    #'GARCH_7':rolling_garch(returns,20*8,1,2,1)
    #'GARCH_8':rolling_garch(returns,20*8,1,3,1)
    #'GARCH_9':rolling_garch(returns,20*8,2,1,1)
    #'GARCH_10':rolling_garch(returns,20*8,3,1,1)
    #'GARCH_11':rolling_garch(returns,20*8,1,1,2)
    #'GARCH_12':rolling_garch(returns,20*8,1,1,3)
    #'GARCH_13':rolling_garch(returns,20*8,4,0,1)
    up=dataset['label'].quantile(0.66)
    down=dataset['label'].quantile(0.33)    
    def S_row (row):
        if row >=up :
            return 3
        elif row <= down :
            return 1
        elif row < up and row > down:
            return 2
    dataset['label'] = dataset['label'].apply(lambda row: S_row(row)) 
    return dataset                 

features=['EMA_1',
'EMA_2',
'EMA_3',
'EMA_4',
'EMA_5',
'MACD_1',
'MACD_2',
'MOM_1',
'MOM_2',
'MOM_3',
'MOM_4',
'VOL_1',
'VOL_2',
'VOL_3',
'VOL_4',
'RSI_2',
'RSI_3',
'RSI_4',
'RSI_5',
'STOC_1',
'STOC_2',
'STOC_3',
'STOC_4',
'STOC_5',
'WR',
'ROC_1',
'ROC_2',
'ROC_3',
'ROC_4',
'ROC_5',
'ATRP_1',
'ATRP_2',
'ATRP_3',
'ATRP_4',
'ATRP_5',
'CCI_1',
'CCI_2',
'CCI_3',
'CCI_4',
'CCI_5',
'DPO_1',
'DPO_2',
'DPO_3',
'DPO_4',
'DPO_5',
'CMO_1',
'CMO_2',
'CMO_3',
'CMO_4',
'CMO_5',
'ADI_1',
'ADI_2',
'ADI_3',
'ADI_4',
'ADI_5',
'DSS_1',
'DSS_2',
'DSS_3',
'DSS_4',
'DSS_5',
'STOCHRSI_1',
'STOCHRSI_2',
'STOCHRSI_3',
'STOCHRSI_4',
'STOCHRSI_5',
'H_1',
'H_2',
'H_3',
'L_1',
'L_2',
'L_3',
'slope_1',
'slope_2',
'slope_3',
'slope_4',
'slope_5',
'slope_6',
'slope_7',
'slope_8',
'slope_9',
'slope_10',
'slope_11',
'slope_12',
'returns',
'GARCH_1',
'GARCH_2',
'GARCH_3'
#'GARCH_4',
#'GARCH_5',
#'GARCH_6',
#'GARCH_7',
#'GARCH_8',
#'GARCH_9',
#'GARCH_10',
#'GARCH_11',
#'GARCH_12',
#'FFT_1',
#"EMD_1",
#"EMD_2",
#"EMD_3",
#"EMD_4",
#"EMD_5",
#"EMD_6",
#"EMD_7",
#"EMD_8",
#"EMD_9",
#"EMD_10",
#"EMD_11",
#"EMD_12",
#"EMD_13",
#"EMD_14",
#"EMD_15",
#"EMD_16",
#"EMD_17"
]

def feature_selection(dataset,features,n):
    model = RFC()
    #model = ExtraTreesClassifier()
    X=dataset[features]
    Y=dataset['label']
    model.fit(X, Y)
    feats = {} # a dict to hold feature_name: feature_importance
    for feature, importance in zip(features, model.feature_importances_):
        feats[feature] = importance #add the name/value pair 
    importances = pd.DataFrame.from_dict(feats, orient='index').rename(columns={0: 'Gini-importance'})
    choose_features=importances.sort_values(by='Gini-importance').tail(n)
    print(importances.sort_values(by='Gini-importance'))
    return choose_features.reset_index()['index'].values
    
def XGB_train(dataset,choose_features):
    train=dataset
    #val=dataset.iloc[round(dataset.shape[0]*0.9):dataset.shape[0],] 
    X=train[choose_features].values
    y=train[['label']].values
    #X_val=val[choose_features].values
    #y_val=val[['label']].values
    dtrain=xgb.DMatrix(data=X,label=y)
    #dval=xgb.DMatrix(data=X_val,label=y_val)
    #for non-scale data, 30 mins    
    param = {'max_depth': 3, 'gamma': 0.0001, 'eta':0.8, 'silent': 1, 'objective': 'multi:softmax',
             'num_class': 4,'n_estimators': 100,'seed':20} 
    param['nthread'] = 1
    param['eval_metric'] = 'mlogloss'
    evallist = [(dtrain, 'train')]
    num_round =100
    bst = xgb.train(param, dtrain, num_round, evallist)
    return bst

def XGB_predict(test,bst_model,choose_features):
    X=xgb.DMatrix(test[choose_features].values)
    pred=bst_model.predict(X, ntree_limit=bst_model.best_ntree_limit)
    return pred

    
def opt_XGB(train,test,features_list):
    score={'features':[0],'accuracy':[0]}
    from sklearn.metrics import accuracy_score
    a,b,c=0.0,0.0,0
    while True:       
        if max(score['accuracy'])<=1.1 and c<3:
            for i in range(10):
                choose_features=feature_selection(dataset=train,features=features_list,n=20) #choose features  
                bst_model=XGB_train(train,choose_features)
                pred=XGB_predict(test,bst_model,choose_features) 
                matrix=table_matrix(test['label'],pred)
                a=(matrix[0,0]+matrix[2,2])/(matrix[0,0]+matrix[2,2]+matrix[0,2]+matrix[2,0])     
                b=accuracy_score(test['label'],pred)
                score['features'].append(choose_features)
                score['accuracy'].append(a+b)
            c+=1
        else:
            print('next data point')
            break
    position_=max(score['accuracy'])
    index_=[i for i,x in enumerate(score['accuracy']) if x == position_]    
    features=score['features'][index_[0]]
    print(position_)
    return features

def rolling_XGB_test(dataset,train,validate,test,features):
    pred=np.repeat(np.nan,test.shape[0])
    train_len=train.shape[0]
    validate_len=validate.shape[0]
    test_len=test.shape[0]
    dataset.reset_index(drop=True,inplace=True)
    for i in range(0,test.shape[0]):
        temp_validate=dataset.loc[(train_len+i):(train_len+validate_len+i),]
        temp_validate.reset_index(drop=True,inplace=True)
        temp_train=dataset.loc[0:(train_len+i)]
        temp_train.reset_index(drop=True,inplace=True)
        temp_test=dataset.loc[(train_len+validate_len+i):(train_len+validate_len+test_len)]
        temp_test.reset_index(drop=True,inplace=True)
        choose_features=opt_XGB(temp_train,temp_validate,features)
        bst_model=XGB_train(temp_train,choose_features)#train
        pred[i]=XGB_predict(temp_test,bst_model,choose_features)[0]
    return pred
        
def backtest(dataset,pred,i):
    table=pd.DataFrame(np.zeros(shape=(dataset.shape[0],3)))
    table.columns=[["StartPrice_0","EndPrice_0","position_0"]]
    table['times']=dataset['times'].values
    table['open']=dataset['open'].values
    table['CumCap_0']=10000
    table.dropna(inplace=True)
    table.reset_index(inplace=True)
    label=0
    a=[0]     
    for j in range(1,table.shape[0]-1):
        label=pred[j]-2                      
        if (label == 1) & (table.loc[j-1,'position_0'][0]==0):
            print('open + position')
            table.loc[(j+1):table.shape[0],"StartPrice_0"]=table.loc[j+1,'open'].values[0]
            table.loc[(j):table.shape[0],"position_0"]=1
        elif (label == -1) & (table.loc[j-1,'position_0'][0]==0):
            print('open - position')
            table.loc[(j+1):table.shape[0],"StartPrice_0"]=table.loc[j+1,'open'].values[0]
            table.loc[(j):table.shape[0],"position_0"]=-1               
        elif (label == 0) & (table.loc[j-1,'position_0'][0] !=0) | (i==table.shape[0]):
            print('close all position')
            table.loc[(j):table.shape[0],"EndPrice_0"]=table.loc[j+1,'open'].values[0]
            table.loc[(j):table.shape[0],"CumCap_0"]=table.loc[j-1,'CumCap_0'].values[0]*(table.loc[j,'EndPrice_0'].values[0]/table.loc[j,'StartPrice_0'].values[0])**(table.loc[j-1,"position_0"].values[0])-table.loc[j-1,'CumCap_0'].values[0]*0.0001  
            #table.loc[(j):table.shape[0],"CumCap_0"]=table.loc[j-1,'CumCap_0']+10000*20*(table.loc[j,'EndPrice_0']/table.loc[j,'StartPrice_0'])**(table.loc[j-1,"position_0"])-10000*20-10000*20*(0.01/table.loc[j,'StartPrice_0']+0.01/table.loc[j,'EndPrice_0'])
            a.append(1)
            table.loc[(j):table.shape[0],"position_0"]=0
        elif (label != table.loc[j-1,'position_0'][0]) & (table.loc[j-1,'position_0'][0] != 0) & (label != 0):
            print('reverse position')
            table.loc[(j):table.shape[0],"EndPrice_0"]=table.loc[j+1,'open'].values[0]
            table.loc[(j):table.shape[0],"CumCap_0"]=table.loc[j-1,'CumCap_0'].values[0]*(table.loc[j,'EndPrice_0'].values[0]/table.loc[j,'StartPrice_0'].values[0])**(table.loc[j-1,"position_0"].values[0])-table.loc[j-1,'CumCap_0'].values[0]*0.0001   
            #table.loc[(j):table.shape[0],"CumCap_0"]=table.loc[j-1,'CumCap_0']+10000*20*(table.loc[j,'EndPrice_0']/table.loc[j,'StartPrice_0'])**(table.loc[j-1,"position_0"])-10000*20-10000*20*(0.01/table.loc[j,'StartPrice_0']+0.01/table.loc[j,'EndPrice_0'])    
            a.append(1)
            table.loc[(j+1):table.shape[0],"StartPrice_0"]=table.loc[j+1,'open'].values[0]
            table.loc[(j):table.shape[0],"position_0"]= label
        else:
            print('do nothing')
        print(table.shape[0]-j)    
    return table,a

In [ ]:
dataset=pd.DataFrame()
dataset=dataset.assign(**{'times':read_data(CU,'open')['times'].values,'open':read_data(CU,'open')['open'].values,
                          'high':read_data(CU,'high')['high'].values,'low':read_data(CU,'low')['low'].values,
                          'close':read_data(CU,'close')['close'].values})
dataset=dataset.dropna()

In [ ]:
dataset.index=dataset['times']
dataset=data_prep(dataset)
dataset=dataset.dropna()

train=dataset.iloc[0:(dataset.shape[0]-120),]
validate=dataset.iloc[(dataset.shape[0]-120):(dataset.shape[0]-80),] #one week  #40 datapoints
test=dataset.iloc[(dataset.shape[0]-80):(dataset.shape[0]),]

pred=rolling_XGB_test(dataset,train,validate,test,features)    
matrix=table_matrix(test['label'],pred)
print((matrix[0,0] + matrix[2,2])/(matrix[0,0] + matrix[2,2] + matrix[0,2] + matrix[2,0]))
from sklearn.metrics import accuracy_score
accuracy_score(test['label'],pred)

result,a=backtest(test,pred,0)
result['CumCap_0'].plot(title='80 hrs cumulative PNL')